In [3]:
import sys; sys.path.insert(0, '../..') # add parent folder path where lib folder is
from utils import helper, config, rayer, kaggle_dataset_helper

In [4]:
"""Example using LightGBM, performing randomized search with TuneSearchCV.
Example taken from https://mlfromscratch.com/gridsearch-keras-sklearn/#/
"""

import lightgbm as lgb
from tune_sklearn import TuneSearchCV
import ray
from ray import tune
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import pandas as pd

from ml.models import common


In [5]:
rayer.get_global_cluster()

2022-12-02 21:00:28,367	INFO packaging.py:527 -- Creating a file package for local directory '/mnt/c/Users/rwmas/GitHub/xai/python-asd/xai/'.
2022-12-02 21:00:59,106	INFO packaging.py:354 -- Pushing file package 'gcs://_ray_pkg_e4bd7b270ec72108.zip' (30.53MiB) to Ray cluster...
2022-12-02 21:01:00,190	INFO packaging.py:367 -- Successfully pushed file package 'gcs://_ray_pkg_e4bd7b270ec72108.zip'.


In [9]:

# Load breast cancer dataset
cancer = load_breast_cancer()
X = cancer.data
y = cancer.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)





In [ ]:
model = lgb.LGBMClassifier()

In [6]:
ds_train, ds_test = kaggle_dataset_helper.get_transaction_predictions_dataset()

ds_train = common.label_encode(ds_train)
ds_test = common.label_encode(ds_test)

ds_train = ds_train.fillna(-1)
ds_test = ds_test.fillna(-1)

df_X = ds_train.loc[:, ds_train.columns != 'target']
df_y = ds_train['target']

X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.33)# , random_state=config.rand_state)


ss = StandardScaler()
X_train_scalar = pd.DataFrame(ss.fit_transform(X_train), columns = X_train.columns)
X_test_scalar = pd.DataFrame(ss.fit_transform(X_test), columns = X_test.columns)

In [7]:
X_train_id = ray.put(X_train_scalar)
y_train_id = ray.put(y_train)
X_test_id = ray.put(X_test_scalar)
y_test_id = ray.put(y_test)

In [ ]:
param_dists = {
    "n_estimators": [400, 700, 1000],
    "colsample_bytree": [0.7, 0.8],
    "max_depth": [15, 20, 25],
    "num_leaves": [50, 100, 200],
    "reg_alpha": [1.1, 1.2, 1.3],
    "reg_lambda": [1.1, 1.2, 1.3],
    "min_split_gain": [0.3, 0.4],
    "subsample": [0.7, 0.8, 0.9],
    "subsample_freq": [20]
}
gs = TuneSearchCV(model, param_dists, n_trials=100, scoring="accuracy")
gs.fit(X_train, y_train)


In [10]:
param_dists  = {
    'n_estimators': tune.randint(400, 700),
    'reg_alpha': tune.loguniform(1e-4, 1e-1),
    'reg_lambda': tune.loguniform(1e-2, 1e-1),
}


gs = TuneSearchCV(model, param_dists, n_trials=100, scoring="accuracy", search_optimization ='hyperopt', cv=3)
gs.fit(X_train_scalar, y_train)


ValueError: Expected array-like (array or non-string sequence), got ClientObjectRef(00ffffffffffffffffffffffffffffffffffffff0100000003000000)

In [ ]:
# print(gs.cv_results_)

pred = gs.predict(X_test)
correct = 0
for i in range(len(y_test)):
    if pred[i] == y_test[i]:
        correct += 1
print("Accuracy:", correct / len(pred))

Accuracy: 0.9736842105263158
